In [1]:
import numpy as np

from air_hockey_challenge.framework import AirHockeyChallengeWrapper, ChallengeCore
from air_hockey_challenge.utils.tournament_agent_wrapper import TrainingTournamentAgentWrapper
from baseline.baseline_agent.baseline_agent import BaselineAgent

from air_hockey_agent.agent_builder import build_agent
from air_hockey_agent.reward import *

from mushroom_rl.utils.dataset import compute_J

In [2]:
mdp = AirHockeyChallengeWrapper(
    env='tournament',
    custom_reward_function=RewardList(
        rewards=[ScoreReward(), ConstraintReward('ee_constr'), ConstraintReward('link_constr')],
        weights=[1, 1, 1],
    ),
)

In [3]:
agent_1 = build_agent(mdp.env_info, layer_sizes=(128, 64, 128))

In [4]:
agent = TrainingTournamentAgentWrapper(
    mdp.env_info,
    agent_1=agent_1,
    agent_2=BaselineAgent(mdp.env_info, 2),
)

In [5]:
core = ChallengeCore(agent, mdp, is_tournament=True, init_state=mdp.base_env.init_state, time_limit=0.02)

In [6]:
initial_replay_size = 5000
n_steps = 4000
n_steps_test = 1000
n_epochs = 20

train_id = 1

In [ ]:
core.learn(n_steps=initial_replay_size, n_steps_per_fit=initial_replay_size)
epoch = 0
while True:
    if epoch % 20 == 0:
        agent_1.save(f'checkpoints/{train_id}/epoch{epoch}.msh')
        print(f'Saved model to checkpoints/{train_id}/epoch{epoch}.msh')

    print(f'{epoch=}')
    print('Training')
    core.learn(n_steps=n_steps, n_steps_per_fit=1)
    print('Validation')
    dataset = agent.get_dataset_1(core.evaluate(n_steps=n_steps_test))

    J = np.mean(compute_J(dataset, mdp.info.gamma))
    R = np.mean(compute_J(dataset))
    E = agent_1.sac.policy.entropy(np.array(list(map(lambda x: x[0], dataset))))

    print(f'{J=}, {R=}, {E=}')

    epoch += 1

Saved model to checkpoints/1/epoch0.msh
epoch=0
Training


 58%|█████▊    | 2310/4000 [01:20<01:01, 27.54it/s]

In [9]:
dataset = core.evaluate(n_steps=1000, render=True)

NameError: name 'exit' is not defined